In [169]:
import os
os.chdir("/Users/fatimarahimi/Desktop/Computational-Geometry")

import math
from ipywidgets import interact, widgets, Layout
import plotly.graph_objects as go
import ast
from sympy import Matrix
import numpy as np
from math import sqrt
from sympy import symbols, Eq, solve

In [171]:
def list_coord(lis, t):
    """
    Make a function that takes a list of points in the plane and a number t (that can be either
    0 or 1) and uses list comprehension to return the list of the t-coordinates of the points.
    """

    return [point[t] for point in lis]

def list_by_two_lists_insertion_sort(orglis, complisone, complistwo):
    """
   takes any type of list and makes a new list that is the result of
   sorting the original list by a list of numbers, and secondarily by a second list of numbers in case
   of a tie in the first list of numbers, using Insertion Sort. The original list should not be changed.
    """
    # make copies of the input lists to keep the original lists unchanged
    newlis = orglis[:]
    primary = complisone[:]
    secondary = complistwo[:]

    # apply Insertion Sort based on the primary and secondary lists
    n = len(newlis)
    for i in range(1, n):
        # save the current elements to be compared and inserted
        current_item = newlis[i]
        current_primary = primary[i]
        current_secondary = secondary[i]

        j = i - 1
        # use the primary list for ordering, and secondary list for tie-breaking
        while j >= 0 and (primary[j] > current_primary or
                          (primary[j] == current_primary and secondary[j] > current_secondary)):
            newlis[j + 1] = newlis[j]
            primary[j + 1] = primary[j]
            secondary[j + 1] = secondary[j]
            j -= 1

        # place the current element at its sorted position
        newlis[j + 1] = current_item
        primary[j + 1] = current_primary
        secondary[j + 1] = current_secondary

    return newlis

def list_bottom_left(points):
    """
    Finds the bottom-left point in a list of points and returns it along with a list of sorted remaining points.
    """
    # Extract y-coordinates (primary) and x-coordinates (secondary) from the points
    y_coords = list_coord(points, 1)
    x_coords = list_coord(points, 0)

    # Sort the points using insertion sort
    sorted_points = list_by_two_lists_insertion_sort(points, y_coords, x_coords)

    # Identify the bottom-left point and remaining points
    bottom_left = sorted_points[0]
    remaining_points = sorted_points[1:]

    return [bottom_left, remaining_points]

def cos_polar_angle(p1, p2):
    """
    Calculates the negative cosine of the polar angle between the vector
    formed by p1 and p2 and the positive x-axis.
    Returns: float: The negative cosine of the polar angle.
    """
    # Convert points to vectors
    vector = Matrix([p2[0] - p1[0], p2[1] - p1[1]])
    unit_vector = Matrix([1, 0])  # Unit vector in the direction of the positive x-axis

    # Compute the dot product and magnitudes
    dot_product = vector.dot(unit_vector)
    vector_magnitude = vector.norm()

    # Compute the cosine of the polar angle
    cos_theta = dot_product / vector_magnitude

    return -cos_theta

def distance(p1, p2):
    """
    Calculate the Euclidean distance between two points.
    """
    dx = p2[0] - p1[0]
    dy = p2[1] - p1[1]
    return math.sqrt(dx**2 + dy**2)

def list_polar_angle_sort(points):
    """
    Ranks points in the plane by increasing polar angle with the bottom-left point,
    and secondarily by increasing distance from the bottom-left point.
    Uses the list_by_two_lists_insertion_sort for sorting.
    Returns:list: The bottom-left point followed by the ranked points.
    """
    # Step 1: Find the bottom-left point
    bottom_left, remaining_points = list_bottom_left(points)

    # Step 2: Calculate the negative cosine of polar angles and distances using list comprehension
    neg_cosines = [cos_polar_angle(bottom_left, p) for p in remaining_points]
    distances = [distance(bottom_left, p) for p in remaining_points]

    # Step 3: Sort the remaining points using list_by_two_lists_insertion_sort
    sorted_points = list_by_two_lists_insertion_sort(remaining_points, neg_cosines, distances)

    # Step 4: Return the bottom-left point and the sorted list
    return [bottom_left, sorted_points]


In [173]:
def parse_points(s: str):
    """Safe parser for a list of (x, y) tuples typed into the Text widget."""
    try:
        pts = ast.literal_eval(s)
        if not isinstance(pts, (list, tuple)):
            raise ValueError("Parsed value is not a list/tuple.")
        return [tuple(map(float, p)) for p in pts]
    except Exception as e:
        raise ValueError(
            "Couldn't parse points. Use format like [(x1, y1), (x2, y2)]. "
            f"Error: {e}"
        )

In [175]:
def make_plotly_figure(points, bottom_left, sorted_points, xrange, yrange):
    xs, ys = zip(*points) if points else ([], [])

    fig = go.Figure()

    # all points (blue) + label next to each
    fig.add_trace(
        go.Scatter(
            x=xs, y=ys,
            mode="markers+text",
            text=[str(p) for p in points],
            textposition="top right",
            marker=dict(size=8),
            name="Points"
        )
    )

    # bottom-left (red, larger)
    fig.add_trace(
        go.Scatter(
            x=[bottom_left[0]], y=[bottom_left[1]],
            mode="markers",
            marker=dict(size=12),
            name="Bottom-left"
        )
    )

    # green lines from bottom-left to each sorted point
    for p in sorted_points:
        fig.add_trace(
            go.Scatter(
                x=[bottom_left[0], p[0]],
                y=[bottom_left[1], p[1]],
                mode="lines",
                line=dict(width=2),
                showlegend=False
            )
        )

    fig.update_layout(
        width=650, height=650,
        margin=dict(l=10, r=10, t=30, b=10),
        xaxis=dict(range=[xrange[0], xrange[1]], zeroline=False),
        yaxis=dict(range=[yrange[0], yrange[1]], zeroline=False),
        title="Polar-angle sort (interactive)"
    )
    fig.update_yaxes(scaleanchor="x", scaleratio=1)  # equal scaling

    return fig

@interact(
    points=widgets.Text(
        value="[(10, 6), (8, 2), (2, 6), (10, 8), (4, 11), (7, 8), (7, 5), (4, 2), (5, 8), (3, 6)]",
        description="Points:",
        layout=Layout(width='90%')
    ),
    xrange=widgets.IntRangeSlider(value=(0, 20), min=-20, max=30, step=1, description="X-Range"),
    yrange=widgets.IntRangeSlider(value=(0, 20), min=-20, max=30, step=1, description="Y-Range"),
)
def interactive_plot(points, xrange, yrange):
    """
    Plot points and the order in which they are sorted.
    """
    try:
        pts = parse_points(points)
    except Exception as e:
        print(e)
        return

    bottom_left, sorted_points = list_polar_angle_sort(pts)

    fig = make_plotly_figure(pts, bottom_left, sorted_points, xrange, yrange)
    fig.show()

    print("Bottom Left Point:", bottom_left)
    print("Sorted Points by Polar Angle and Distance:", sorted_points)

interactive(children=(Text(value='[(10, 6), (8, 2), (2, 6), (10, 8), (4, 11), (7, 8), (7, 5), (4, 2), (5, 8), …

In [177]:
def triangle_orientation(a, b, c):
    """
    Determine the orientation of the triangle.
    Returns:
    1 if counterclockwise,
    -1 if clockwise,
    0 if collinear.
    """
    x0, y0 = a
    x1, y1 = b
    x2, y2 = c
    # Compute the determinant directly
    determinant = (x1 - x0) * (y2 - y0) - (x2 - x0) * (y1 - y0)
    if determinant > 0:
        return 1  # Counterclockwise
    elif determinant < 0:
        return -1  # Clockwise
    else:
        return 0  # Collinear
def point_in_line_segment(a0, a1, p):
    """
    Determines whether a point p lies on the line segment defined by two points a0 and a1.
    Returns:
            1 if p is in the interior of the line segment,
            0 if p is one of the endpoints,
           -1 if p is not on the line segment.
    """
    x0, y0 = a0
    x1, y1 = a1
    c, d = p

   # Check collinearity using the triangle_orientation function
    if triangle_orientation(a0, a1, p) != 0:
        return -1  # Not collinear, so not on the line segment

    # Check if the point is within the bounds of the segment
    if min(x0, x1) <= c <= max(x0, x1) and min(y0, y1) <= d <= max(y0, y1):
        # Check if it's an endpoint
        if (c, d) == a0 or (c, d) == a1:
            return 0  # Point is an endpoint
        else:
            return 1  # Point is in the interior of the segment
    else:
        return -1  # Collinear but outside the bounds of the segment

def triangle_unsigned_area(a, b, c):
    """
    Calculate the unsigned area of a triangle given three vertices.
    Returns:The unsigned area of the triangle.
    """
    x0, y0 = a
    x1, y1 = b
    x2, y2 = c
    return abs((x0 * (y1 - y2) + x1 * (y2 - y0) + x2 * (y0 - y1)) / 2.0)

def point_in_triangle(a0, a1, a2, p):
    """
    Determines whether a point p lies in the triangle defined by three points a0, a1, and a2.
    Returns:
        0 if p is one of the vertices,
        2 if p is in the interior of one of the edges,
        1 if p is in the interior of the triangle,
       -1 if p is in the exterior of the triangle.
    """
    # Check if the point is one of the vertices
    if p == a0 or p == a1 or p == a2:
        return 0
    # Check if the point lies on any of the edges
    if point_in_line_segment(a0, a1, p) == 1 or point_in_line_segment(a1, a2, p) == 1 or point_in_line_segment(a2, a0, p) == 1:
        return 2
    # Check if the point is in the interior using triangle orientation
    orientation1 = triangle_orientation(a0, a1, p)
    orientation2 = triangle_orientation(a1, a2, p)
    orientation3 = triangle_orientation(a2, a0, p)
    # If the point is on the same side for all edges, it's inside the triangle
    if orientation1 == orientation2 == orientation3:
        return 1
    # Otherwise, the point is outside the triangle
    return -1

In [179]:
def parse_point_single(s: str):
    """Reuse parse_points for a single (x,y) by wrapping it in a list."""
    pts = parse_points(f"[{s}]")
    if len(pts) != 1:
        raise ValueError("Enter exactly one point, e.g. (6,6)")
    return pts[0]

In [181]:
def make_triangle_figure(a0, a1, a2, p, xrange, yrange, inside_code):
    # triangle polygon (closed)
    tri_x = [a0[0], a1[0], a2[0], a0[0]]
    tri_y = [a0[1], a1[1], a2[1], a0[1]]

    fig = go.Figure()

    # triangle fill
    fig.add_trace(go.Scatter(
        x=tri_x, y=tri_y, mode="lines",
        fill="toself", line=dict(width=2),
        name="Triangle"
    ))

    # vertices (green)
    vx = [a0[0], a1[0], a2[0]]
    vy = [a0[1], a1[1], a2[1]]
    fig.add_trace(go.Scatter(
        x=vx, y=vy, mode="markers+text",
        text=[f"a0{a0}", f"a1{a1}", f"a2{a2}"],
        textposition="top right",
        marker=dict(size=10),
        name="Vertices"
    ))

    # point p (red if outside/edge, purple if inside, orange if vertex)
    color_map = {1: "purple", 0: "orange", 2: "red", -1: "red"}
    fig.add_trace(go.Scatter(
        x=[p[0]], y=[p[1]], mode="markers+text",
        text=[f"p{p}"], textposition="bottom left",
        marker=dict(size=12),
        name="Point p"
    ))

    fig.update_layout(
        width=650, height=650,
        margin=dict(l=10, r=10, t=30, b=10),
        xaxis=dict(range=[xrange[0], xrange[1]], zeroline=False),
        yaxis=dict(range=[yrange[0], yrange[1]], zeroline=False),
        title="Point-in-Triangle (interactive)"
    )
    fig.update_traces(selector=dict(name="Point p"),
                      marker=dict(color=color_map.get(inside_code, "red")))
    fig.update_yaxes(scaleanchor="x", scaleratio=1)  # equal aspect
    return fig

In [183]:
@interact(
    vertices=widgets.Text(
        value="(8, 2), (6, 10), (2, 8)",
        description="Vertices (a0,a1,a2):",
        layout=Layout(width="90%")
    ),
    point=widgets.Text(
        value="(6,6)",
        description="Point p:",
        layout=Layout(width="50%")
    ),
    xrange=widgets.IntRangeSlider(value=(0, 15), min=-20, max=20, step=1, description="X-Range"),
    yrange=widgets.IntRangeSlider(value=(0, 15), min=-20, max=20, step=1, description="Y-Range")
)
def interactive_plot(vertices, point, xrange, yrange):
    """Interactive triangle plot using your functions unchanged."""
    try:
        a0, a1, a2 = parse_points(vertices)   # <-- your parser for list of points
        p = parse_point_single(point)         # <-- wraps and reuses your parser
    except Exception as e:
        print("Parse error:", e)
        return

    # use YOUR classifier
    code = point_in_triangle(a0, a1, a2, p)

    # use YOUR figure builder
    fig = make_triangle_figure(a0, a1, a2, p, xrange, yrange, code)
    fig.show()

    # same messages as before
    if code == 1:
        print(f"The point {p} is in the interior of the triangle with vertices {a0}, {a1}, and {a2}.")
    elif code == 0:
        print(f"The point {p} is one of the vertices of the triangle with vertices {a0}, {a1}, and {a2}.")
    elif code == 2:
        print(f"The point {p} is on an edge of the triangle with vertices {a0}, {a1}, and {a2}.")
    else:
        print(f"The point {p} is NOT in the triangle with vertices {a0}, {a1}, and {a2}.")


interactive(children=(Text(value='(8, 2), (6, 10), (2, 8)', description='Vertices (a0,a1,a2):', layout=Layout(…

In [185]:
def circumcircle(a0, a1, a2):
    """
    Finds the center and radius of the circumcircle of the triangle defined by three points.
    Returns:
        tuple: Center and radius of the circumcircle, or 0 if the triangle is degenerate.
    """
    # Extract coordinates
    x0, y0 = a0
    x1, y1 = a1
    x2, y2 = a2

    # Define the matrices
    q_matrix = Matrix([[x0, y0, 1],
                       [x1, y1, 1],
                       [x2, y2, 1]])
    g_matrix = Matrix([[x0**2 + y0**2, y0, 1],
                       [x1**2 + y1**2, y1, 1],
                       [x2**2 + y2**2, y2, 1]])
    h_matrix = Matrix([[x0**2 + y0**2, x0, 1],
                       [x1**2 + y1**2, x1, 1],
                       [x2**2 + y2**2, x2, 1]])
    f_matrix = Matrix([[x0**2 + y0**2, x0, y0],
                       [x1**2 + y1**2, x1, y1],
                       [x2**2 + y2**2, x2, y2]])

    # Calculate the determinants
    q = q_matrix.det()
    if q == 0:
        return 0  # The triangle is degenerate

    g = -g_matrix.det()
    h = h_matrix.det()
    f = -f_matrix.det()

    # Calculate center and radius
    m = -g / (2 * q)
    n = -h / (2 * q)
    r = ((g**2 + h**2) / (4 * q**2) - f / q)**0.5

    return (float(m), float(n)), float(r)

In [187]:
def point_in_circle(a0, a1, a2, p):
    """
    Returns:
        1: Point p is inside the circumcircle.
        0: Point p is on the circumcircle.
       -1: Point p is outside the circumcircle.
        2: Triangle is degenerate (no circumcircle).
    """
    # Build the determinant matrix
    t1 = Matrix([
        [(p[0]**2) + (p[1]**2), p[0], p[1], 1],
        [(a0[0]**2) + (a0[1]**2), a0[0], a0[1], 1],
        [(a1[0]**2) + (a1[1]**2), a1[0], a1[1], 1],
        [(a2[0]**2) + (a2[1]**2), a2[0], a2[1], 1]
    ])
    t = t1.det()

    # Check if the triangle is degenerate
    q1 = Matrix([
        [a0[0], a0[1], 1],
        [a1[0], a1[1], 1],
        [a2[0], a2[1], 1]
    ])
    q = q1.det()

    # Degenerate triangle check
    if triangle_orientation(a0, a1, a2) == 0:
        return 2  # Degenerate triangle, no circumcircle

    # Point position relative to circumcircle
    if t == 0:
        return 0  # On the boundary
    elif q * t < 0:
        return 1  # Inside
    else:  # q * t > 0
        return -1  # Outside

In [189]:
def circle_xy(center, radius, num=400):
    """Generate x,y points for a circle."""
    cx, cy = center
    t = np.linspace(0, 2*np.pi, num)
    return cx + radius*np.cos(t), cy + radius*np.sin(t)

In [191]:
@interact(
    points=widgets.Text(
        value="[(8, 2), (6, 10), (2, 8)]",
        description="Triangle Points:",
        layout=Layout(width="90%")
    ),
    xrange=widgets.IntRangeSlider(value=(0, 15), min=-20, max=30, step=1, description="X-Range"),
    yrange=widgets.IntRangeSlider(value=(0, 15), min=-20, max=30, step=1, description="Y-Range")
)
def interactive_plot(points, xrange, yrange):
    # Parse input
    try:
        pts = parse_points(points)
        if len(pts) != 3:
            print("Please enter exactly 3 points, e.g. (8,2), (6,10), (2,8)")
            return
    except Exception as e:
        print("Parse error:", e)
        return

    # Compute circumcircle
    result = circumcircle(pts[0], pts[1], pts[2])

    if result == 0:
        # Degenerate triangle: just draw the triangle
        tri_x = [pts[0][0], pts[1][0], pts[2][0], pts[0][0]]
        tri_y = [pts[0][1], pts[1][1], pts[2][1], pts[0][1]]
        fig = go.Figure()
        fig.add_trace(go.Scatter(x=tri_x, y=tri_y, mode="lines",
                                 fill="toself", line=dict(width=2), name="Triangle"))
        fig.add_trace(go.Scatter(x=[p[0] for p in pts], y=[p[1] for p in pts],
                                 mode="markers+text",
                                 text=[str(p) for p in pts],
                                 textposition="top right",
                                 marker=dict(size=10), name="Vertices"))
        fig.update_layout(
            width=650, height=650, margin=dict(l=10, r=10, t=30, b=10),
            xaxis=dict(range=[xrange[0], xrange[1]], zeroline=False),
            yaxis=dict(range=[yrange[0], yrange[1]], zeroline=False),
            title="Circumcircle (degenerate triangle)"
        )
        fig.update_yaxes(scaleanchor="x", scaleratio=1)
        fig.show()
        print("The triangle is degenerate and has no circumcircle.")
        return

    center, radius = result
    m, n = center

    # Build Plotly figure
    tri_x = [pts[0][0], pts[1][0], pts[2][0], pts[0][0]]
    tri_y = [pts[0][1], pts[1][1], pts[2][1], pts[0][1]]
    circ_x, circ_y = circle_xy(center, radius)

    fig = go.Figure()
    # Triangle
    fig.add_trace(go.Scatter(x=tri_x, y=tri_y, mode="lines",
                             fill="toself", line=dict(width=2), name="Triangle"))
    # Vertices
    fig.add_trace(go.Scatter(x=[p[0] for p in pts], y=[p[1] for p in pts],
                             mode="markers+text",
                             text=[str(p) for p in pts],
                             textposition="top right",
                             marker=dict(size=10), name="Vertices"))
    # Circumcircle
    fig.add_trace(go.Scatter(x=circ_x, y=circ_y, mode="lines",
                             line=dict(width=2), name="Circumcircle"))
    # Center
    fig.add_trace(go.Scatter(x=[m], y=[n], mode="markers+text",
                             text=[f"center{center}"],
                             textposition="bottom left",
                             marker=dict(size=12), name="Center"))

    fig.update_layout(
        width=650, height=650, margin=dict(l=10, r=10, t=30, b=10),
        xaxis=dict(range=[xrange[0], xrange[1]], zeroline=False),
        yaxis=dict(range=[yrange[0], yrange[1]], zeroline=False),
        title="Triangle & Circumcircle (Plotly)"
    )
    fig.update_yaxes(scaleanchor="x", scaleratio=1)  # equal aspect
    fig.show()

    print(f"Center of Circumcircle: {center}")
    print(f"Radius of Circumcircle: {radius}")

interactive(children=(Text(value='[(8, 2), (6, 10), (2, 8)]', description='Triangle Points:', layout=Layout(wi…

In [193]:
@interact(
    triangle_points=widgets.Text(
        value="[(8, 2), (6, 10), (2, 8)]",
        description="Triangle Points:",
        layout=Layout(width="90%")
    ),
    test_point=widgets.Text(
        value="(9,6)",
        description="Test Point:",
        layout=Layout(width="50%")
    ),
    xrange=widgets.IntRangeSlider(value=(0, 15), min=-30, max=30, step=1, description="X-Range"),
    yrange=widgets.IntRangeSlider(value=(0, 15), min=-30, max=30, step=1, description="Y-Range")
)
def interactive_plot(triangle_points, test_point, xrange, yrange):
    try:
        a0, a1, a2 = parse_points(triangle_points)
        p = parse_point_single(test_point)
    except Exception as e:
        print("Parse error:", e)
        return

    #classify point relative to circumcircle
    result = point_in_circle(a0, a1, a2, p)

    #handle degenerate triangle early
    if result == 2:
        tri_x = [a0[0], a1[0], a2[0], a0[0]]
        tri_y = [a0[1], a1[1], a2[1], a0[1]]
        fig = go.Figure()
        fig.add_trace(go.Scatter(x=tri_x, y=tri_y, mode="lines",
                                 fill="toself", line=dict(width=2), name="Triangle"))
        fig.add_trace(go.Scatter(x=[a0[0], a1[0], a2[0]], y=[a0[1], a1[1], a2[1]],
                                 mode="markers+text", text=[f"a0{a0}", f"a1{a1}", f"a2{a2}"],
                                 textposition="top right", marker=dict(size=10), name="Vertices"))
        fig.add_trace(go.Scatter(x=[p[0]], y=[p[1]], mode="markers+text",
                                 text=[f"P{p}"], textposition="bottom left",
                                 marker=dict(size=12), name="Test Point"))
        fig.update_layout(width=650, height=650, margin=dict(l=10, r=10, t=30, b=10),
                          xaxis=dict(range=[xrange[0], xrange[1]], zeroline=False),
                          yaxis=dict(range=[yrange[0], yrange[1]], zeroline=False),
                          title="Degenerate triangle (no circumcircle)")
        fig.update_yaxes(scaleanchor="x", scaleratio=1)
        fig.show()
        print("The triangle is degenerate and has no circumcircle.")
        return

    #compute circumcircle 
    cc = circumcircle(a0, a1, a2)
    if cc == 0:
        # safety: if circumcircle returns 0 counts degenerate as well
        print("The triangle is degenerate and has no circumcircle.")
        return
    center, radius = cc
    center = (float(center[0]), float(center[1]))
    radius = float(radius)

    #Plotly figure
    tri_x = [a0[0], a1[0], a2[0], a0[0]]
    tri_y = [a0[1], a1[1], a2[1], a0[1]]
    circ_x, circ_y = circle_xy(center, radius)

    fig = go.Figure()
    # Triangle
    fig.add_trace(go.Scatter(x=tri_x, y=tri_y, mode="lines",
                             fill="toself", line=dict(width=2), name="Triangle"))
    # Vertices
    fig.add_trace(go.Scatter(x=[a0[0], a1[0], a2[0]], y=[a0[1], a1[1], a2[1]],
                             mode="markers+text", text=[f"a0{a0}", f"a1{a1}", f"a2{a2}"],
                             textposition="top right", marker=dict(size=10), name="Vertices"))
    # Circumcircle
    fig.add_trace(go.Scatter(x=circ_x, y=circ_y, mode="lines",
                             line=dict(width=2), name="Circumcircle"))
    # Test point (color by result)
    color_map = {0: "orange", 1: "purple", -1: "red"}  # boundary, inside, outside
    fig.add_trace(go.Scatter(x=[p[0]], y=[p[1]], mode="markers+text",
                             text=[f"P{p}"], textposition="bottom left",
                             marker=dict(size=12, color=color_map.get(result, "red")),
                             name="Test Point"))

    fig.update_layout(
        width=650, height=650, margin=dict(l=10, r=10, t=30, b=10),
        xaxis=dict(range=[xrange[0], xrange[1]], zeroline=False),
        yaxis=dict(range=[yrange[0], yrange[1]], zeroline=False),
        title="Point vs. Circumcircle"
    )
    fig.update_yaxes(scaleanchor="x", scaleratio=1)
    fig.show()

    if result == 0:
        print(f"The point {p} is on the boundary of circle.")
    elif result == 1:
        print(f"The point {p} is on the interior of circle.")
    elif result == -1:
        print(f"The point {p} is outside of the circle.")


interactive(children=(Text(value='[(8, 2), (6, 10), (2, 8)]', description='Triangle Points:', layout=Layout(wi…

In [195]:
def convex_hull(points):
    """
    Computes the convex hull of a set of points in the plane using the Graham Scan algorithm.
    Returns: list: Vertices of the convex hull in counterclockwise order.
    """
    if len(points) < 3:
        return points  # Convex hull is the same as the input if there are fewer than 3 points

    # Step 1: Find the bottom-left point and sort the remaining points
    botl, remaining_points = list_bottom_left(points)
    sorted_points = list_polar_angle_sort(points)[1]  # Second item is the sorted points

    # Step 2: Initialize the convex hull
    hull = [botl, sorted_points[0]]

    # Step 3: Process the remaining points
    for p in sorted_points[1:]:
        hull.append(p)
        # Ensure counterclockwise turn
        while len(hull) >= 3 and triangle_orientation(hull[-3], hull[-2], hull[-1]) != 1:
            hull.pop(-2)  # Remove the second-to-last point if not turning left

    return hull

In [197]:
def convex_hull_triangulation(points):
    """
    Computes a list of triangles for a convex hull triangulation of the given points in the plane.
    Returns: list: A list of triangles, where each triangle is a list of three vertices (points).
    """

    # Step 1: Compute the convex hull
    chlis = convex_hull(points)

    # Step 2: Prepare the triangulation list
    chtri = []

    # Step 3: Create triangles using the convex hull vertices
    for i in range(2, len(chlis)):
        # Each triangle is formed by the first point and two consecutive points
        triangle = [chlis[0], chlis[i - 1], chlis[i]]
        chtri.append(triangle)

    return chtri

In [199]:
def triangle_splitting_triangulation(lis):
    """
    Perform a triangle-splitting triangulation of the points in the given list.
    Returns:
        list: A list of triangles of a triangle-splitting triangulation.
    """
    # Step 1: Compute the convex hull
    chlis = convex_hull(lis)

    # Step 2: Compute the convex hull triangulation
    chtri = convex_hull_triangulation(lis)

    # Step 3: Determine the interior points
    intlis = [p for p in lis if p not in chlis]

    # Step 4: Initialize tstri with the convex hull triangulation
    tstri = chtri[:]

    # Step 5: Process each interior point
    for point in intlis:
        for triangle in tstri:
            a, b, c = triangle

            # Check if the point lies inside the triangle
            pos = point_in_triangle(a, b, c, point)

            if pos == 1:  # Point is in the interior of the triangle
                # Remove the original triangle and add three new triangles
                tstri.remove(triangle)
                tstri.extend([[point, a, b], [point, b, c], [point, c, a]])
                break

            elif pos == 2:  # Point is on the interior of an edge
                # Identify the edge and its opposite triangles
                edge_triangles = [
                    tri for tri in tstri
                    if point_in_line_segment(tri[0], tri[1], point) == 1 or
                       point_in_line_segment(tri[1], tri[2], point) == 1 or
                       point_in_line_segment(tri[2], tri[0], point) == 1
                ]
                #If intlis[i] is in the interior of an edge [d, e] of two triangles [d, e, f] and [d, e, g]
#of tstri, then remove [d, e, f] and [d, e, g] from tstri,
                if len(edge_triangles) == 2:
                    tri1, tri2 = edge_triangles
                    d, e = [point for point in tri1 if point in tri2]  # Find the common edge
                    f = [v for v in tri1 if v not in [d, e]][0]
                    g = [v for v in tri2 if v not in [d, e]][0]

                    # Remove the two triangles and add four new triangles
                    tstri.remove(tri1)
                    tstri.remove(tri2)
                    tstri.extend([[point, d, f], [point, e, f], [point, d, g], [point, e, g]])
                break

    return tstri

In [201]:
@interact(
    points=widgets.Text(
        value="[(10, 6), (8, 2), (2, 6), (10, 10), (4, 11), (7, 8), (7, 5), (4, 2), (5, 8), (3, 6)]",
        description="Points:",
        layout=Layout(width='90%')
    ),
    show_option=widgets.RadioButtons(
        options=["Show Original Points", "Show Convex Hull"],
        value="Show Convex Hull",
        description="Display Option:"
    ),
    xrange=widgets.IntRangeSlider(value=(0, 20), min=-30, max=30, step=1, description="X-Range"),
    yrange=widgets.IntRangeSlider(value=(0, 20), min=-30, max=30, step=1, description="Y-Range")
)
def interactive_plot(points, show_option, xrange, yrange):
    try:
        pts = parse_points(points)  # expects "[(x,y), ...]"
    except Exception as e:
        print("Parse error:", e)
        return

    fig = go.Figure()
    fig.add_trace(go.Scatter(
        x=[p[0] for p in pts],
        y=[p[1] for p in pts],
        mode="markers",
        marker=dict(size=8),
        name="Points"
    ))

    hull = None

    if show_option == "Show Convex Hull":
        # compute convex hull 
        hull = convex_hull(pts)  # should return vertices in order
        if hull and len(hull) >= 3:
            hx = [p[0] for p in hull] + [hull[0][0]]
            hy = [p[1] for p in hull] + [hull[0][1]]
            fig.add_trace(go.Scatter(
                x=hx, y=hy,
                mode="lines",
                line=dict(width=4),
                name="Convex Hull"
            ))
        else:
            print("Convex hull could not be computed (need at least 3 non-collinear points).")

    fig.update_layout(
        width=650, height=650,
        margin=dict(l=10, r=10, t=30, b=10),
        xaxis=dict(range=[xrange[0], xrange[1]], zeroline=False),
        yaxis=dict(range=[yrange[0], yrange[1]], zeroline=False),
        title="Points / Convex Hull (Plotly)"
    )
    fig.update_yaxes(scaleanchor="x", scaleratio=1)

    fig.show()

    print("Points:", str(pts))
    if show_option == "Show Convex Hull" and hull is not None:
        print("List of Convex Hull Vertices:", str(hull))


interactive(children=(Text(value='[(10, 6), (8, 2), (2, 6), (10, 10), (4, 11), (7, 8), (7, 5), (4, 2), (5, 8),…

In [203]:
def edges_from_triangles(triangles):
    """Converts list of triangles [[a,b,c],...] into a set of undirected edges."""
    edges = set()
    for a, b, c in triangles:
        for u, v in [(a,b), (b,c), (c,a)]:
            edge = tuple(sorted(((round(u[0],6), round(u[1],6)),
                                 (round(v[0],6), round(v[1],6)))))
            edges.add(edge)
    return edges

def edges_to_xy(edges):
    """Converts edge set into x,y lists (with None separators)."""
    xs, ys = [], []
    for (x1,y1), (x2,y2) in edges:
        xs += [x1, x2, None]
        ys += [y1, y2, None]
    return xs, ys

In [205]:
@interact(
    points=widgets.Text(
        value="[(10,6),(8,2),(2,6),(10,10),(4,11),(7,8),(7,5),(4,2),(5,8),(3,6)]",
        description="Points:",
        layout=Layout(width='90%')
    ),
    show_option=widgets.RadioButtons(
        options=["Show Vertices","Show Convex Hull","Show Convex Hull Triangulation","Show Triangle-Splitting Triangulation"],
        value="Show Triangle-Splitting Triangulation",
        description="Display Option:",
        layout=Layout(width='90%')
    ),
    xrange=widgets.IntRangeSlider(value=(0,20), min=-30, max=30, step=1, description="X-Range"),
    yrange=widgets.IntRangeSlider(value=(0,20), min=-30, max=30, step=1, description="Y-Range")
)
def interactive_plot(points, show_option, xrange, yrange):
    try:
        pts = parse_points(points)
    except Exception as e:
        print("Parse error:", e)
        return
    
    fig = go.Figure()
    
    # always show points
    fig.add_trace(go.Scatter(
        x=[p[0] for p in pts],
        y=[p[1] for p in pts],
        mode="markers",
        marker=dict(size=8, color="blue"),
        name="Points"
    ))
    
    hull = None
    triangles = []
    
    if show_option in ("Show Convex Hull","Show Convex Hull Triangulation","Show Triangle-Splitting Triangulation"):
        hull = convex_hull(pts)
    
    if show_option == "Show Convex Hull Triangulation":
        triangles = convex_hull_triangulation(pts)
    
    if show_option == "Show Triangle-Splitting Triangulation":
        triangles = triangle_splitting_triangulation(pts)
    
    if triangles:
        edges = edges_from_triangles(triangles)
        ex, ey = edges_to_xy(edges)
        fig.add_trace(go.Scatter(
            x=ex, y=ey, mode="lines",
            line=dict(width=2, color="red"),
            name="Triangulation"
        ))
    
    if hull and len(hull) >= 3:
        hx = [p[0] for p in hull] + [hull[0][0]]
        hy = [p[1] for p in hull] + [hull[0][1]]
        fig.add_trace(go.Scatter(
            x=hx, y=hy, mode="lines",
            line=dict(width=3, color="green"),
            name="Convex Hull"
        ))
    
    fig.update_layout(
        width=700, height=700,
        margin=dict(l=10,r=10,t=30,b=10),
        xaxis=dict(range=[xrange[0],xrange[1]], zeroline=False),
        yaxis=dict(range=[yrange[0],yrange[1]], zeroline=False),
        title=show_option
    )
    fig.update_yaxes(scaleanchor="x", scaleratio=1)
    fig.show()
    
    print("Points:", pts)
    if hull: print("Convex Hull Vertices (CCW):", hull)
    if triangles: print(f"{show_option} Triangles:", triangles)


interactive(children=(Text(value='[(10,6),(8,2),(2,6),(10,10),(4,11),(7,8),(7,5),(4,2),(5,8),(3,6)]', descript…

In [207]:
def delaunay_triangulation(points):
    """
    Perform Delaunay triangulation on a set of points in the plane using the Edge Flipping Algorithm.
    Returns:
        list: List of triangles in the Delaunay triangulation.
    """
    # Step 1: Compute the convex hull of the points
    chlis = convex_hull(points)

    # Step 2: Compute the convex hull triangulation
    chtri = convex_hull_triangulation(points)

    # Step 3: Compute the triangle-splitting triangulation
    tstri = triangle_splitting_triangulation(points)

    # Step 4: Initialize dtri as the triangle-splitting triangulation
    dtri = tstri[:]

    while True:
        # Step 5: Compute the list of all edges in dtri, ensuring no duplicates
        edge = compute_edges(dtri)
        print('edge while = ', edge)

        # Step 6: Identify interior edges shared by exactly two triangles
        intedge = identify_interior_edges(edge, dtri)

        # Step 7: Check legality of each interior edge
        illegal_edge_found = check_and_flip_edges(intedge, dtri)

        # If no illegal edge was found, the triangulation is complete
        if not illegal_edge_found:
            break

    return dtri

In [209]:
def compute_edges(dtri):
    """
    Compute all unique edges in the given list of triangles.
    """
    edge = [] # Initialize an empty list to store unique edges
    for triangle in dtri:
       # Extract the vertices of the triangle
        a, b, c = triangle
         # Form the edges of the triangle
        edges = [[a, b], [b, c], [c, a]]
        for e in edges:
          # Add the edge to the list if it is not already present (in either order)
            if e not in edge and e[::-1] not in edge:
                edge.append(e)
    return edge

In [211]:
def identify_interior_edges(edges, dtri):
    """
    Identify edges shared by exactly two triangles.
    """
    # Create a dictionary to map each edge to the triangles containing it
    edge_triangle_map = {tuple(sorted(e)): [] for e in edges}
    for triangle in dtri:
      # Extract the vertices of the triangle
        a, b, c = triangle
         # Form the edges of the triangle
        edgess = [[a, b], [b, c], [c, a]]
        for e in edgess:
           # Use sorted tuples to ensure edge uniqueness
            e_sorted = tuple(sorted(e))
            edge_triangle_map[e_sorted].append(triangle)
    # Extract edges that are shared by exactly two triangles: list(e) is converting tuple e into a list and sort it and it use list comprehension to filter all the edges
    #triangle in edge_triangle_map.items() is filtering through the triangles in the dictionary and incluse the ones that has a shared edge
    intedge = [list(e) for e, triangles in edge_triangle_map.items() if len(triangles) == 2]
    return intedge

In [213]:
def check_and_flip_edges(intedge, dtri):
    """
    Check each interior edge and flip it if necessary.
    Returns:
        bool: True if an illegal edge was found and flipped, False otherwise.
    """
    for e in intedge:
        [a, b] = e
        # Get the two triangles sharing this edge
        triangles = [t for t in dtri if all(v in t for v in e)]
        if len(triangles) != 2:
            continue

        [t1, t2] = triangles
        # Determine the vertices opposite to the edge
        x = [v for v in t1 if v not in e][0]
        y = [v for v in t2 if v not in e][0]

        # Check if the edge is illegal using circumcircle criterion
        if point_in_circle(a, b, x, y) == 1 or point_in_circle(a, b, y, x) == 1:
            # Edge is illegal, perform edge flipping
            dtri.remove(t1)
            dtri.remove(t2)
            dtri.append([a, x, y])
            dtri.append([b, x, y])
            return True
    return False

In [215]:
def edges_from_triangles(triangles):
    """Return set of undirected edges from list of triangles [[a,b,c], ...]."""
    edges = set()
    for a, b, c in triangles:
        for u, v in [(a, b), (b, c), (c, a)]:
            edge = tuple(sorted(((float(u[0]), float(u[1])), (float(v[0]), float(v[1])))))
            edges.add(edge)
    return edges

def edges_to_xy(edges):
    """Convert edge set into x,y lists for Plotly line plotting (with None separators)."""
    xs, ys = [], []
    for (x1, y1), (x2, y2) in edges:
        xs += [x1, x2, None]
        ys += [y1, y2, None]
    return xs, ys

def circle_xy(center, radius, n=200):
    """Generate x,y for a circle (center=(cx,cy), radius)."""
    cx, cy = float(center[0]), float(center[1])
    t = np.linspace(0, 2*np.pi, n)
    return cx + radius*np.cos(t), cy + radius*np.sin(t)

In [217]:
@interact(
    points=widgets.Text(
        value="[(10, 6), (8, 2), (2, 6), (10, 10), (4, 11), (7, 8), (7, 5), (4, 2), (5, 8), (3, 6)]",
        description="Points:", layout=Layout(width='90%')
    ),
    display_option=widgets.RadioButtons(
        options=[
            "Show Original Points",
            "Show Convex Hull",
            "Show Convex Hull Triangulation",
            "Triangle-Splitting Triangulation",
            "Show Delaunay Triangulation",
            "Show Delaunay Triangulation with Circles"
        ],
        value="Show Delaunay Triangulation",
        description="View"
    ),
    xrange=widgets.IntRangeSlider(value=(0, 12), min=-10, max=20, step=1, description="X-Range"),
    yrange=widgets.IntRangeSlider(value=(0, 12), min=-10, max=20, step=1, description="Y-Range"),
)
def interactive_plot(points, display_option, xrange, yrange):
    try:
        pts = parse_points(points)
    except Exception as e:
        print("Parse error:", e)
        return

    fig = go.Figure()

    # draw original points
    fig.add_trace(go.Scatter(
        x=[p[0] for p in pts],
        y=[p[1] for p in pts],
        mode="markers",
        marker=dict(size=8, color="blue"),
        name="Points"
    ))

    # Convex Hull
    if display_option in ("Show Convex Hull","Show Convex Hull Triangulation","Triangle-Splitting Triangulation","Show Delaunay Triangulation","Show Delaunay Triangulation with Circles"):
        hull = convex_hull(pts)
        if hull and len(hull) >= 3:
            hx = [p[0] for p in hull] + [hull[0][0]]
            hy = [p[1] for p in hull] + [hull[0][1]]
            fig.add_trace(go.Scatter(
                x=hx, y=hy, mode="lines",
                line=dict(width=3, color="green"),
                name="Convex Hull"
            ))
        else:
            hull = None
    else:
        hull = None

    # Convex Hull Triangulation
    if display_option == "Show Convex Hull Triangulation":
        chtri = convex_hull_triangulation(pts)
        edges = edges_from_triangles(chtri)
        ex, ey = edges_to_xy(edges)
        fig.add_trace(go.Scatter(x=ex, y=ey, mode="lines",
                                 line=dict(width=2, color="blue"),
                                 name="CH Triangulation"))

    # Triangle-Splitting
    if display_option == "Triangle-Splitting Triangulation":
        tstri = triangle_splitting_triangulation(pts)
        edges = edges_from_triangles(tstri)
        ex, ey = edges_to_xy(edges)
        fig.add_trace(go.Scatter(x=ex, y=ey, mode="lines",
                                 line=dict(width=2, color="blue"),
                                 name="TS Triangulation"))

    # Delaunay
    if display_option in ("Show Delaunay Triangulation","Show Delaunay Triangulation with Circles"):
        dtri = delaunay_triangulation(pts)
        edges = edges_from_triangles(dtri)
        ex, ey = edges_to_xy(edges)
        fig.add_trace(go.Scatter(x=ex, y=ey, mode="lines",
                                 line=dict(width=2, color="orange"),
                                 name="Delaunay"))
        if display_option == "Show Delaunay Triangulation with Circles":
            for tri in dtri:
                cc = circumcircle(tri[0], tri[1], tri[2])
                if cc != 0:
                    center, r = cc
                    cx, cy = circle_xy(center, r)
                    fig.add_trace(go.Scatter(
                        x=cx, y=cy, mode="lines",
                        line=dict(width=1, color="orange", dash="dot"),
                        name="Circumcircle",
                        showlegend=False
                    ))

    fig.update_layout(
        width=750, height=750,
        margin=dict(l=10,r=10,t=30,b=10),
        xaxis=dict(range=[xrange[0], xrange[1]], zeroline=False),
        yaxis=dict(range=[yrange[0], yrange[1]], zeroline=False),
        title=display_option
    )
    fig.update_yaxes(scaleanchor="x", scaleratio=1)
    fig.show()

    print("Points:", pts)
    if hull: print("Convex Hull:", hull)
    if display_option == "Show Convex Hull Triangulation":
        print("Convex Hull Triangulation:", chtri)
    if display_option == "Triangle-Splitting Triangulation":
        print("Triangle-Splitting Triangulation:", tstri)
    if display_option.startswith("Show Delaunay"):
        print("Delaunay Triangulation:", dtri)


interactive(children=(Text(value='[(10, 6), (8, 2), (2, 6), (10, 10), (4, 11), (7, 8), (7, 5), (4, 2), (5, 8),…

In [219]:
def is_strictly_convex(points):
  '''
  Method takes in a list of points in the convex hull and returns true if the hull is strictly convex and false else
  '''

  for i in range(len(points)):
    if triangle_orientation(points[i-2], points[i-1], points[i]) != 1:
      return False
  return True

In [221]:
def angle_bisector(e1, e2):
    '''
    Takes two adjacent edges and finds the edge along the angle bisector they form.
    Returns a line segment originating from v2 parallel to the bisector of v2
    '''
    # Extract the shared point
    v2 = [p for p in e1 if p in e2][0]

    v1 = Matrix(e1[0])
    v2 = Matrix(e1[1])
    v3 = Matrix(e2[1])

    # Calculate the direction vectors of the two sides of the angle
    vec1 = v1 - v2
    vec2 = v3 - v2

    # Normalize the vectors to unit vectors
    norm_vec1 = sqrt(sum(i**2 for i in vec1))
    norm_vec2 = sqrt(sum(i**2 for i in vec2))

    unit_vec1 = vec1 / norm_vec1
    unit_vec2 = vec2 / norm_vec2

    # Calculate the angle bisector direction vector
    bisector = unit_vec1 + unit_vec2

    # Calculate the endpoint of the edge along the bisector
    bisector_endpoint = v2 + bisector

    return [tuple(v2), tuple(bisector_endpoint)]

In [223]:
def line_intersection(edge1, edge2):
    '''
    Finds the intersection point of two non-parallel edges.
    Returns None if the lines are parallel or coincident.
    '''

    # Extract points from edges
    P1, P2 = edge1
    P3, P4 = edge2

    # Extract coordinates
    a0, a1 = P1
    b0, b1 = P2
    c0, c1 = P3
    d0, d1 = P4

    # Handle vertical lines by checking for undefined slopes
    if a0 == b0:  # Edge 1 is vertical
        x_intersection = a0
        if c0 == d0:  # Edge 2 is also vertical
            return None  # Parallel vertical lines
        slope2 = (d1 - c1) / (d0 - c0)
        y_intersection = slope2 * (x_intersection - c0) + c1
    elif c0 == d0:  # Edge 2 is vertical
        x_intersection = c0
        slope1 = (b1 - a1) / (b0 - a0)
        y_intersection = slope1 * (x_intersection - a0) + a1
    else:
        # Calculate slopes
        slope1 = (b1 - a1) / (b0 - a0)
        slope2 = (d1 - c1) / (d0 - c0)

        # Check for parallel lines
        if slope1 == slope2:
            return None

        # Form equations for line 1 and line 2
        x, y = symbols('x y')
        eq1 = Eq(y - a1, slope1 * (x - a0))
        eq2 = Eq(y - c1, slope2 * (x - c0))

        # Solve the equations
        solution = solve((eq1, eq2), (x, y))
        x_intersection = float(solution[x])
        y_intersection = float(solution[y])

    return (round(x_intersection,3), round(y_intersection,3))


In [225]:
def orthogonal_distance(point, line_segment):
    """
    Calculate the perpendicular distance from a point to a line segment.
    This method uses the triangle area formula A = (1/2) * |det| and computes
    the distance as A divided by the length of the segment.
    Returns:
    - float: The shortest distance from the point to the line segment.
    """
    # Compute the length of the segment
    segment_vector = np.array(line_segment[1]) - np.array(line_segment[0])
    segment_length = np.linalg.norm(segment_vector)

    # Compute the twice area of the triangle formed by the point and the segment
    x0, y0 = point
    x1, y1 = line_segment[0]
    x2, y2 = line_segment[1]
    twice_area = abs((x1 * (y2 - y0) + x2 * (y0 - y1) + x0 * (y1 - y2)))

    # Compute and return the perpendicular distance
    return twice_area / segment_length

In [227]:
def does_line_intersect_segment(line_point1, line_point2, seg_point1, seg_point2):
    """
    Determines the relationship between a line and a line segment.
    Returns:
    - 1: If the infinite line intersects the interior of the segment.
    - 0: If the line contains one or both endpoints of the segment.
    - -1: If the line does not intersect the segment.
    """
    # Check collinearity with segment endpoints
    orientation1 = triangle_orientation(line_point1, line_point2, seg_point1)
    orientation2 = triangle_orientation(line_point1, line_point2, seg_point2)

    if orientation1 == 0 or orientation2 == 0:
        # The line passes through one or both endpoints
        return 0
    elif orientation1 == orientation2:
        # The segment lies entirely on one side of the line
        return -1
    else:
        # The line intersects the segment
        return 1

In [229]:
def medial_axis(points):
  '''
  Takes a list of edges of the convex hull and returns a list of edges of the medial axis
  '''
  # Error
  if points == 0 or points == None:
    return 0

  # Error handling when points are not strictly convex
  if not is_strictly_convex(points):
    return 1

  # Edges in ccw order
  edges = [[points[i-1], points[i]] for i in range(len(points))]

  # Base case
  # If length of points is 3 then return the three edges in the medial axis of the triangle
  if len(points) == 3:
    incenter = line_intersection(angle_bisector(edges[0], edges[1]), angle_bisector(edges[1], edges[2])) # Finds the incenter of the triangle

    # Finds the edges of the medial axis of triangle stores them in ccw
    medaxis = [[points[0], incenter], [points[1], incenter], [points[2], incenter]]

    return medaxis # Returns the list of edges in the medial axis

  # Recursive case
  else:
    point_dict = {} # Dictionary of edges as keys to points
    for i in range(len(edges)):
      point_dict[tuple(edges[i-1])] = line_intersection(angle_bisector(edges[i-2], edges[i-1]), angle_bisector(edges[i-1], edges[i]))

    height_dict = {} # Dictionary of edges as keys to height
    for edge in edges:
      height_dict[tuple(edge)] = orthogonal_distance(point_dict[tuple(edge)], edge)

    min_edge = min(height_dict, key=height_dict.get) # Edge corresponding to smallest height

    # Stores the adjacent edges in the convex hull to be removed then added back in after recursion
    for i in range(len(edges)):
      if list(min_edge) == edges[i-1]:
        left_edge = edges[i-2] # Edge that comes before the min edge
        right_edge = edges[i] # Edge that comes after min edge

    # Error handling when the edges angles do not sum to less than pi
    left_vector = Matrix(left_edge[0]) - Matrix(left_edge[1])
    right_vector = Matrix(right_edge[1]) - Matrix(right_edge[0])
    if triangle_orientation(tuple(left_vector), (0,0), tuple(right_vector)) != 1:
      return 0

    # Stores the edges to be added to the medial axis
    p = point_dict[min_edge] # The point associated with the shortest height
    medial_edge1 = [p, min_edge[0]]
    medial_edge2 = [p, min_edge[1]]

    # Point where left and right edge intersect
    leftright_intersection = line_intersection(left_edge, right_edge)

    # Find new lines which are extensions of left edge and right edge
    # Keeps ccw ordering
    new_left_edge = [[p for p in left_edge if p not in min_edge][0], leftright_intersection]
    new_right_edge = [leftright_intersection, [p for p in right_edge if p not in min_edge][0]]

    # Replace left and right edge with the extended lines
    # This keeps ccw order
    new_edges = edges[:] # Copy of edges to be manipulated
    for i in range(len(new_edges)):
      if new_edges[i] == left_edge:
        new_edges[i] = new_left_edge
      if new_edges[i] == right_edge:
        new_edges[i] = new_right_edge

    # Removes min_edge
    new_edges.remove(list(min_edge))

    # Now new_edges is a polygon with one fewer edge
    # Recursive call to this new polygon
    pts_from_new_edges = [edge[0] for edge in new_edges]
    medial_axis_edges = medial_axis(pts_from_new_edges)

    # Error handling if the algorithm failed in some recursive step earlier
    if medial_axis_edges == 0 or medial_axis_edges == None:
      return 0

    # Add the medial axis edges found earlier into the medial axis
    medial_axis_edges.append(medial_edge1)
    medial_axis_edges.append(medial_edge2)

    # Find the edge in the medial axis list that we want to remove
    for hulledge in edges:
      for axisedge in medial_axis_edges:
        # If the interior of an edge in the medial axis intersects a hull edge then it is the one that needs to be removed
        # There will only be one of these by construction
        if does_line_intersect_segment(hulledge[0], hulledge[1], axisedge[0], axisedge[1]) == 1:
          edge_to_remove = axisedge

    # Remove axis edge that is
    medial_axis_edges.remove(list(edge_to_remove))

    # Find vertex of edge to remove that is in the polygon
    interior_point = edge_to_remove[0]
    for axisedge in medial_axis_edges:
      for point in axisedge:
        if list(point) == list(edge_to_remove[1]):
          interior_point = edge_to_remove[1]

    # Creates new edge to be added to the medial axis
    final_edge_to_insert = [interior_point, p]

    medial_axis_edges.append(final_edge_to_insert)

    return medial_axis_edges

In [231]:
def edges_to_xy(edges):
    """Converts a list of edges [[p1,p2],...] into x,y lists with None separators"""
    xs, ys = [], []
    for a, b in edges:
        xs += [a[0], b[0], None]
        ys += [a[1], b[1], None]
    return xs, ys

@interact(
    points=widgets.Text(
        value="[(2,5),(10,5),(12,7),(8,10),(2,11)]",
        description="Points:", layout=Layout(width='90%')
    ),
    xrange=widgets.IntRangeSlider(value=(0, 15), min=-10, max=20, step=1, description="X Range"),
    yrange=widgets.IntRangeSlider(value=(0, 15), min=-10, max=20, step=1, description="Y Range")
)
def interactive_medial_axis(points, xrange, yrange):
    try:
        lis = parse_points(points)
    except Exception as e:
        print("Parse error:", e)
        return

    result = medial_axis(lis)

    # Error handling
    if result == -2:
        print("Error: The polygon is not strictly convex.")
        return
    elif result == -1:
        print("Error: Extended edges meet on the wrong side.")
        return
    elif result == 0:
        print("Error: Extended edges are parallel.")
        return

    fig = go.Figure()

    # Polygon outline (blue)
    px = [p[0] for p in lis] + [lis[0][0]]
    py = [p[1] for p in lis] + [lis[0][1]]
    fig.add_trace(go.Scatter(
        x=px, y=py, mode="lines+markers",
        line=dict(color="blue", width=3),
        marker=dict(size=8, color="blue"),
        name="Polygon"
    ))

    # Medial axis edges (red)
    if result:
        ex, ey = edges_to_xy(result)  # result should be list of [p1,p2]
        fig.add_trace(go.Scatter(
            x=ex, y=ey, mode="lines",
            line=dict(color="red", width=2),
            name="Medial Axis"
        ))

    # Layout
    fig.update_layout(
        width=700, height=700,
        margin=dict(l=10, r=10, t=30, b=10),
        xaxis=dict(range=[xrange[0], xrange[1]], zeroline=False),
        yaxis=dict(range=[yrange[0], yrange[1]], zeroline=False),
        title="Medial Axis"
    )
    fig.update_yaxes(scaleanchor="x", scaleratio=1)
    fig.show()

    print("Medial Axis Edges:", result)


interactive(children=(Text(value='[(2,5),(10,5),(12,7),(8,10),(2,11)]', description='Points:', layout=Layout(w…